<a href="https://colab.research.google.com/github/rich-hyun/Kuggle_9th_team_/blob/main/jaehyunkim_NVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# 데이터 로딩
df = pd.read_csv('https://raw.githubusercontent.com/rich-hyun/Kuggle_9th_team_/main/reviews_data.csv')

# 데이터프레임의 첫 번째 행과 처음 다섯 열을 출력
print(df.iloc[0, :5])


name                                                    Helen
location                                    Wichita Falls, TX
Date                                  Reviewed Sept. 13, 2023
Rating                                                    5.0
Review      Amber and LaDonna at the Starbucks on Southwes...
Name: 0, dtype: object
